In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import pinv
import pandas as pd

np.set_printoptions(precision=4, suppress=True)

import mdpy as mdp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def matrix_series(A, n):
    mdp.is_square(A)
    ret = np.zeros_like(A)
    mat = np.eye(len(A))
    for i in range(n):
        ret += mat
        mat = mat @ A
    return ret

# Problem Setup

Here we define an MDP and solve it analytically, computing both the expected return (i.e., the value function `v_pi`) and its variance (`v_var`, using Sobel's approach) for each state.

In [3]:
# MDP solved analytically
ns = 6
I = np.eye(ns)

# Probability of transitioning from state s_i --> s_j = P[i,j]
P = np.diag(np.ones(ns-1), 1) * 0.5
P[:,0] = 0.5
P[-1, 0] = 1

# Expected reward for transitioning from s_i --> s_j = R[i,j]
R = np.zeros((ns, ns))
# -1 Reward for non-terminal transitions
R[:,:] = -1
# Reaching edge has zero reward
R[-2, -1] = 0
# Transitions from terminal state have zero reward
R[-1,:] = 0
r = np.sum(P*R, axis=1)

# State-dependent discount
gvec = np.ones(ns)*0.9
gvec[0] = 0
G = np.diag(gvec)

# State-dependent bootstrapping
lvec = np.ones(ns)*0.0
L = np.diag(lvec)

# Value function (expected Monte Carlo return)
v_pi = pinv(I - P @ G) @ r

# Compute stationary distribution for transition matrix
d_pi = mdp.stationary(P)
D = np.diag(d_pi)


# From Sobel, setting up variance Bellman equation
T = -v_pi**2
for i in range(ns):
    for j in range(ns):
        T[i] += P[i,j] * (R[i,j] + gvec[j]*v_pi[j])**2

# Alternatively,
# T = np.sum(P * (R + G @ v_pi)**2, axis=1) - v_pi**2
        
# Solve Bellman equation for variance of return
v_var = pinv(I - P @ G @ G) @ T 

# print(T)
print('v_pi:\n', v_pi)
print('per-state variance:\n', v_var)

v_pi:
 [-1.7641 -1.6981 -1.5513 -1.225  -0.5    -0.    ]
per-state variance:
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]


# Trying It Out Empirically

Empirically checking via simulation is useful for finding errors.
The code for simulating the MDP is somewhat inefficient but simple to write and debug.

We use the same MDP as above. 

Note that the use of state-dependent γ allows for an effectively episodic problem without requiring us to actually break the trajectory into episodes.

The trajectory is kept as a list of dictionaries that record all relevant data for each time step.
We then compute additional quantities, the *return* and the *squared return*, taking the expression for the latter from the VTD paper (White & White).

In [4]:
def compute_return(history):
    ret = []
    g = 0
    for step in reversed(history):
        g *= step['gm']
        g += step['r']
        ret.append({'g': g, **step})
    return list(reversed(ret))

def compute_squared_return(history):
    ret = []
    g_sq = 0
    g_next = 0
    for step in reversed(history):
        g_sq *= step['gm']**2
        g_sq += step['r']**2 + 2*step['gm']*step['r']*g_next
        ret.append({'g_sq': g_sq, **step})
        g_next = step['g']
    return list(reversed(ret))

In [5]:
# Simulate our MDP
num_steps = 10000

states = np.arange(ns)
features = [i for i in np.eye(ns)]

# Initial state
s0  = 0
s   = s0
x   = features[s]

history = []
for i in range(num_steps):
    p_next = x @ P
    sp = np.random.choice(states, p=p_next)
    xp = features[sp]
    gm = gvec[sp]
    reward = R[s, sp]
    
    history.append({'s': s, 'sp': sp, 'gm': gm, 'r': reward})
    
    # Next iteration
    s = sp
    x = xp.copy()
    
# Augment the history with the return at each timestep
history = compute_return(history)
history = compute_squared_return(history)

# Convert to pandas dataframe
df = pd.DataFrame(history)

In [6]:
df.head()

,g,g_sq,gm,r,s,sp
0,-3.439,11.826721,0.9,-1.0,0,1
1,-2.710,7.344100,0.9,-1.0,1,2
2,-1.900,3.610000,0.9,-1.0,2,3
3,-1.000,1.000000,0.9,-1.0,3,4
4,0.000,0.000000,0.9,0.0,4,5


In [16]:
# Check expected squared return
grouped = df.groupby('s')
g_sq = np.array(grouped.aggregate({'g_sq': np.mean}))

# Display results
grouped.aggregate({'g':np.mean, 'g_sq': np.mean}).T

s,0,1,2,3,4,5
g_sq,4.008053,3.565755,2.745723,1.595544,0.455657,0.0
g,-1.776638,-1.713232,-1.547019,-1.205360,-0.455657,0.0


In [14]:
# Variance via squared-return minus expected return squared
grouped = df.groupby('s')
a = np.array(grouped.aggregate({'g': np.mean})**2)
b = np.array(grouped.aggregate({'g_sq': np.mean}))
print("Variance via E[G^2] - E[G]^2:\n", np.ravel(b - a))
print("Analytical variance (Sobel):\n", v_var)

Variance via E[G^2] - E[G]^2:
 [ 0.8516  0.6306  0.3525  0.1427  0.248   0.    ]
Analytical variance (Sobel):
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]


In [18]:
# Compare with variance computed directly
print("Variance (via numpy.var):")
grouped = df.groupby('s')
grouped.aggregate({'g': np.var}).T

Variance (via numpy.var):


s,0,1,2,3,4,5
g,0.85178,0.63084,0.352726,0.14287,0.248795,0.0


This suggests that we our formulas were accurate, although more simulations could further improve the accuracy.

# Second Moment of the Return

An alternative way to compute the variance analytically is by defining and solving a Bellman equation for the second moment of the return and using the fact that $\operatorname{Var}[X] = \mathbb{E}[X^2] - \mathbb{E}[X]^2$.

In *A Greedy Approach to Adapting the Trace Parameter for Temporal Difference Learning*, White & White provide just such a Bellman equation, which we use here.

In [21]:
# Using the VTD paper to calculate second moments of the return.
# Note that here we are using the most accurate values for everything
# in order to check the equations.
Pbar = np.zeros((ns, ns))
Rbar = np.zeros((ns,ns))
rbar = np.zeros(ns)

# Specify parameters
lvec = np.ones(ns)

# Calculate R-bar transition matrix
for i in range(ns):
    for j in range(ns):
        Rbar[i,j] = R[i,j]**2 + 2*gvec[j]*lvec[j]*R[i,j]*v_pi[j]

# Calculate r-bar vector
for i in range(ns):
    for j in range(ns):
        rbar[i] += P[i,j]*Rbar[i,j]

# Calculate P-bar
for i in range(ns):
    for j in range(ns):
        Pbar[i,j] += P[i,j]*(gvec[j]**2)*(lvec[j]**2)
        
        
# Calculate second moment of return
r_second = pinv(I - Pbar) @ rbar

# Print the results
print("Second moment of return:\n", r_second)
print("Estimated variance via second moment of return:\n", r_second - v_pi**2)
print("Sobel variance:\n", v_var)


# An alternative approach, which is somewhat more concise
# Second moment of return
rr = (P*R**2) @ np.ones(ns) + (2*P @ G * R) @ v_pi
vv = pinv(I - P @ G @ G)@(rr)

Second moment of return:
 [ 3.9533  3.5187  2.7718  1.6525  0.5     0.    ]
Estimated variance via second moment of return:
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]
Sobel variance:
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]


## Takeaway

The two methods produce the same result, as expected.

## Implementation Notes

We calculated things as carefully as we can (and with as complete information as possible) to avoid mis-specifying things.
Of note is that in a few cases whether the expected reward *matrix* vs. expected reward *vector* is used substantially affects the calculations.
So while there are less verbose ways to solve the problem, the following is perhaps a little cleaner to examine and debug.

# Second Moment of δ-return

Using the equations in the White & White paper, we can calculate the second moment of the δ-return.
The δ-return is the discounted sum of future TD-errors, which we can define via: 

$$
G^{\delta, \lambda}_{t} = \sum_{n=0}^{\infty} \delta_{t+n} \prod_{k=1}^{n-1} \gamma_{t+k} \lambda_{t+k}
$$

Note that the δ-return also encodes the bias (difference between the expected λ-return for each state and the approximate value function).

$$G^{\lambda}_{t} - \hat{v}(S_t) = \sum_{n=0}^{\infty} \delta_{t+n} \prod_{k=1}^{n-1} \gamma_{t+k} \lambda_{t+k}$$

This means that for $\lambda = 1$ we have a way of computing the bias with respect to the Monte Carlo return:

$$
G^{\lambda=1}_{t} - \hat{v}(S_t) = v_{\pi} - \hat{v} 
$$

Disregarding the values of λ used in the approximation process for $\hat{v}$, we can choose alternative values to get the bias with respect to a particular λ-return.

## Sanity Check: Exact Value Function

We check that our algorithm works by computing the second moment of the δ-return for when the approximate and 'true' value functions are identical.
The expected TD-error for each state should be zero, as should the bias (which should be equal to the δ-return).

The second moment of the δ-return may be nonzero.
In fact, it should be equal to both the $\delta^2$-return and the variance, and since the bias is zero, the mean squared-error as well.

In [23]:
# Approximate value function is identical to true value function
v_hat = v_pi

# Bias of approximate value function
bias = v_pi - v_hat

# TD error matrix, for error given transition i-->j
Δ = np.zeros_like(R)
for i in range(ns):
    for j in range(ns):
        Δ[i,j] = (R[i,j] + gvec[j]*v_hat[j] - v_hat[i])

# Expected TD-error
δ = (P*Δ) @ np.ones(ns)
        
# Expected δ^2
δ_sq = (P * Δ**2) @ np.ones(ns)
        
# δ-return
gd = pinv(I - P @ G) @ δ

# δ^2-return
gd_sq = pinv(I - P @ G @ G) @ δ_sq

# Second moment of δ-return
dd = (P * Δ**2) @ np.ones(ns) + (2*P @ G * Δ) @ gd
gd_second = pinv(I - P @ G @ G)@(dd)

print("v_π:\n", v_pi)
print("v_hat:\n", v_hat)
print("bias:\n", v_pi-v_hat)
print("δ-return:\n", gd)
print("δ^2-return:\n", gd_sq)
print("Second moment expected 'reward' (r-bar):\n", dd)
print("Second moment of delta-return:\n", gd_second)
print("Sobel variance:\n", v_var)
print("Expected squared error:\n", (v_pi - v_hat)**2 + v_var)

v_π:
 [-1.7641 -1.6981 -1.5513 -1.225  -0.5    -0.    ]
v_hat:
 [-1.7641 -1.6981 -1.5513 -1.225  -0.5    -0.    ]
bias:
 [ 0.  0.  0.  0.  0.  0.]
δ-return:
 [ 0. -0.  0.  0.  0.  0.]
δ^2-return:
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]
Second moment 'reward' (r-bar):
 [ 0.5839  0.4873  0.3039  0.0506  0.25    0.    ]
Second moment of delta-return:
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]
Sobel variance:
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]
Expected squared error:
 [ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]


As expected, 

## Inaccurate Value Function

In [ ]:
# Approximate value function
# v_hat = v_pi #+ 0.1*np.arange(len(v_hat))
v_hat = v_pi + [0, -0.1, 0.5, 0, 0.0, 0]

# Bias of approximate value function
bias = v_pi - v_hat

# TD error matrix, for error given transition i-->j
Δ = np.zeros_like(R)
for i in range(ns):
    for j in range(ns):
        Δ[i,j] = (R[i,j] + gvec[j]*v_hat[j] - v_hat[i])

# Expected TD-error
δ = (P*Δ) @ np.ones(ns)
        
# Expected δ^2
δ_sq = (P * Δ**2) @ np.ones(ns)
        
# Second moment of δ-return 
rr = (P*R**2) @ np.ones(ns) + (2*P @ G * R) @ v_pi
vv = pinv(I - P @ G @ G)@(rr)

# δ-return
gd = pinv(I - P @ G) @ δ

# δ^2-return
gd_sq = pinv(I - P @ G @ G) @ δ_sq

# Second moment
dd = (P * Δ**2) @ np.ones(ns) + (2*P @ G * Δ) @ gd
gd_second = pinv(I - P @ G @ G)@(dd)

print("v_π:\n", v_pi)
print("v_hat:\n", v_hat)
print("bias:\n", v_pi-v_hat)
print("δ-return:\n", gd)
print("δ^2-return:\n", gd_sq)
print("Second moment 'reward' (r-bar):\n", dd)
print("Second moment of delta-return:\n", gd_second)
print("Sobel variance:\n", v_var)
print("Expected squared error:\n", (v_pi - v_hat)**2 + v_var)

# Variance of δ-return

Given the popularity of the movie *Inception*, it behooves us to go deeper.

On a more serious note, I am not really sure how to interpret this but kept it in anyways because I wish to avoid having to redo it if it turns out to be relevant.

In [19]:
# Approximate value function
# v_hat = v_pi + 0.1*np.arange(len(v_hat))
v_hat = v_pi + [0, -0.1, 0.5, 0, 0.0, 0]


# TD error matrix, for error given transition i-->j
Δ = np.zeros_like(R)
for i in range(ns):
    for j in range(ns):
        Δ[i,j] = (R[i,j] + gvec[j]*v_hat[j] - v_hat[i])

# Expected per-state TD-error
δ = (P*Δ) @ np.ones(ns)
        
# Sobel-like method for variance of δ-return 
T_d = -δ**2
for i in range(ns):
    for j in range(ns):
        T_d[i] += P[i,j] * (Δ[i,j] + gvec[j]*δ[j])**2

# Alternatively,
# T_d = np.sum(P * (Δ + G @ δ)**2, axis=1) - δ**2

# Calculating variance of δ-return
dd = pinv(I - P @ G @ G) @ T_d

print(v_var)
print(dd)
print(v_var - dd)

[ 0.8412  0.6353  0.3654  0.1519  0.25    0.    ]
[ 0.6662  0.5396  0.3654  0.1519  0.25    0.    ]
[ 0.175   0.0956 -0.     -0.     -0.     -0.    ]
